<center> <h1> Features Engineering Codes for fast coding</h1> <center>
    <center> <h2> Auteur: Vinh NGUYEN </h2> <center>

* [Rare labels](#Rare-labels)
* [Missing Indicator](#Missing-indicator)
* [MV Imputer](#MV-Imputer)
    * [SimpleImputer for mean/median](#SimpleImputer-for-mean/median)
    * [Same technique for arbitrary imputation. Strategy = 'constant'](#Same-technique-for-arbitrary-imputation.-Strategy-=-'constant')
    * [Same for most_frequent. We can use most_frequent strat for both num and cat variables](#Same-for-most_frequent.-We-can-use-most_frequent-strat-for-both-num-and-cat-variables)
    * [Automatic best imputation method search](#Automatic-best-imputation-method-search)
* [FeatureEngine](#FeatureEngine)
    * [Using pipeline from sklearn](#Using-pipeline-from-sklearn)
    * [Arbitrary method](#Arbitrary-method)
    * [EndTail method](#EndTail-method)
    * [Add MissingIndicator](#Add-MissingIndicator)
* [Encoding](#Encoding)
    * [One hot encoding](#One-hot-encoding)
    * [One-hot encoding of top categories](#One-hot-encoding-of-top-categories)
    * [Ordinal encoding](#Ordinal-encoding)
    * [Count/Frequency encoding](#Count/Frequency-encoding)
    * [Target guided encoding](#Target-guided-encoding)
    * [Mean encoding](#Mean-encoding)
    * [Probability ratio encoding](#Probability-ratio-encoding)
    * [Weight of evidence encoding](#Weight-of-evidence-encoding)
    * [Compare encoding methods performance on RF](#Compare-encoding-methods-performance-on-RF)
* [Variable transformation](#Variable-transformation)
* [Discretisation](#Discretisation)
* [Outliers handling: only remove from train set](#Outliers-handling:-only-remove-from-train-set)
* [Features scaling: important except for trees](#Features-scaling:-important-except-for-trees)
* [Mixed variables](#Mixed-variables)
* [Date and time variables](#Date-and-time-variables)
    * [Date](#Date)
    * [Time](#Time)
* [Recapitulation: Pipeline](#Recapitulation:-Pipeline)
    * [Add Cross-Validation](#Add-cross-validation)


    
    
    
    
   


# Rare labels

Pb : Overfitting due to the split train/test
<br> **Solution : Groupe rare labels under a new label "rare"**

In [ ]:
### FUNCTION TO CALCULATE:
# 1) the % of houses per category (var)
# 2) the mean target per category


def calculate_mean_target_per_category(df, var):

    # total number of houses
    total_houses = len(df)

    # percentage of houses per category
    temp_df = pd.Series(df[var].value_counts() / total_houses).reset_index()
    temp_df.columns = [var, 'perc_houses']

    # add the mean target
    temp_df = temp_df.merge(df.groupby([var])['target'].mean().reset_index(),
                            on=var,
                            how='left')

    return temp_df


#apply : temp_df = calculate_mean_target_per_category(data, 'Neighborhood')


In [ ]:
## FUNCTION TO PLOT THE CATEGORY FREQUENCY AND THE MEAN TARGET
# This will help us visualise the relationship between the
# target and the labels of the  categorical variable

def plot_categories(df, var):
    
    fig, ax = plt.subplots(figsize=(8, 4))
    plt.xticks(df.index, df[var], rotation=90)

    ax2 = ax.twinx()
    ax.bar(df.index, df["perc_houses"], color='lightgrey')
    ax2.plot(df.index, df["target"], color='green', label='Seconds')
    ax.axhline(y=0.05, color='red')
    ax.set_ylabel('percentage of houses per category')
    ax.set_xlabel(var)
    ax2.set_ylabel('Average target per category')
    plt.show()
    
#apply : plot_categories(temp_df, 'Neighborhood')    

In [ ]:
## FUNCTION TO REPLACE ALL THE LABELS THAT APPEAR IN <5% BY THE LABEL "RARE"

def group_rare_labels(df, var):

    total_houses = len(df)

    # first I calculate the % of houses for each category
    temp_df = pd.Series(df[var].value_counts() / total_houses)

    # now I create a dictionary to replace the rare labels with the
    # string 'rare' if they are present in less than 5% of houses

    grouping_dict = {
        k: ('rare' if k not in temp_df[temp_df >= 0.05].index else k)
        for k in temp_df.index
    }

    # now I replace the rare categories
    tmp = df[var].map(grouping_dict)

    return tmp

# apply : # group rare labels in Neighborhood

#data['Neighborhood_grouped'] = group_rare_labels(data, 'Neighborhood')

#data[['Neighborhood', 'Neighborhood_grouped']].head(10)

# Missing indicator

Add a binary missing indicator may help improve the performance

In [ ]:
# we are going to use the following variables,
# some are categorical some are numerical

cols_to_use = [
    'LotFrontage', 'MasVnrArea', # numerical
    'BsmtQual', 'FireplaceQu', # categorical
    'SalePrice' # target
]

In [ ]:
# let's make a function to add a missing indicator
# binary variable

def missing_indicator(df, variable):    
    return np.where(df[variable].isnull(), 1, 0)

In [ ]:
# let's loop over all the variables and add a binary 
# missing indicator with the function we created

for variable in cols_to_use:
    X_train[variable+'_NA'] = missing_indicator(X_train, variable)
    X_test[variable+'_NA'] = missing_indicator(X_test, variable)
    
X_train.head()

# MV Imputer

## SimpleImputer for mean/median

In [ ]:
# first we need to make lists, indicating which features
# will be imputed with each method

numeric_features_mean = ['LotFrontage']
numeric_features_median = ['MasVnrArea', 'GarageYrBlt']

# then we instantiate the imputers, within a pipeline
# we create one mean imputer and one median imputer
# by changing the parameter in the strategy

numeric_mean_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

numeric_median_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])

# then we put the features list and the transformers together
# using the column transformer

# we need to add remainder = True to indicate what we want
# ALL the columns returned at the end of the transformation
# and not just the engineered ones, which is the default
# behaviour of ColumnTransformer. 

preprocessor = ColumnTransformer(transformers=[
    ('mean_imputer', numeric_mean_imputer, numeric_features_mean),
    ('median_imputer', numeric_median_imputer, numeric_features_median)
], remainder='passthrough')


# parameters of the ColumnTransformer
# remainder = 'passthrough' indicates that we want to retain ALL the columns in the dataset
            # otherwise only those specified in the imputing steps will be kept
    
# for more details follow the sklearn page:
# https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

In [ ]:
# now we fit the preprocessor
preprocessor.fit(X_train)

# and now we can impute the data
X_train = preprocessor.transform(X_train)

# and check it worked
np.mean(np.isnan(X_train))

# and now we can impute the test data
X_test = preprocessor.transform(X_test)

# and check it worked
np.mean(np.isnan(X_test))

# remember that the returned object  is a NumPy array
X_train

In [ ]:
# let's capture the columns in a list

remainder_cols = [cols_to_use[c] for c in [0, 1, 2, 3, 4, 5]]
remainder_cols

In [ ]:
# capture the data back in a dataframe
pd.DataFrame(X_train,
             columns = numeric_features_mean+numeric_features_median+remainder_cols).head()

## Same technique for arbitrary imputation. Strategy = 'constant'

In [ ]:
# first we need to make lists, indicating which features
# will be imputed with each value

features_LotFrontAge = ['LotFrontage']
features_MasVnrArea = ['MasVnrArea']
features_GarageYrBlt = ['GarageYrBlt']

# then we instantiate the imputers, within a pipeline
# we create one imputer per feature
# within the imputer I indicate the arbitrary value
# which is differet for each variable

imputer_LotFrontAge = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 999)),
])

imputer_MasVnrArea = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = -10)),
])

imputer_GarageYrBlt = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 1700)),
])

# then we put the features list and the transformers together
# using the column transformer

# in this example, I will use the default parameter of ColumnTransformer
# remainder = drop, which means that only the imputed features will
# be retained, and the rest dropped

preprocessor = ColumnTransformer(transformers=[
    ('imputer_LotFrontAge', imputer_LotFrontAge, features_LotFrontAge),
    ('imputer_MasVnrArea', imputer_MasVnrArea, features_MasVnrArea),
    ('imputer_GarageYrBlt', imputer_GarageYrBlt, features_GarageYrBlt)
],remainder = 'drop')

## Same for most_frequent. We can use most_frequent strat for both num and cat variables

In [ ]:
# first we need to make lists, indicating which features
# will be imputed with each method

features_numeric = ['BsmtUnfSF', 'LotFrontage', 'MasVnrArea', ] 
features_categoric = ['BsmtQual', 'FireplaceQu', 'MSZoning',
                      'Street', 'Alley']

# then we instantiate the imputers, within a pipeline
# we create one mean imputer and one frequent category imputer
# by changing the parameter in the strategy

numeric_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

categoric_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])

# then we put the features list and the transformers together
# using the column transformer

preprocessor = ColumnTransformer(transformers=[
    ('numeric_imputer', numeric_imputer, features_numeric),
    ('categoric_imputer', categoric_imputer, features_categoric)
])

In [ ]:
# To see stratistics for frequent category imputer

preprocessor.named_transformers_['categoric_imputer'].named_steps['imputer'].statistics_

## Automatic best imputation method search

In [ ]:
# We create the preprocessing pipelines for both
# numerical and categorical data

# adapted from Scikit-learn code available here under BSD3 license:
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, features_numerical),
        ('categorical', categorical_transformer, features_categorical)])

# Note that to initialise the pipeline I pass any argument to the transformers.
# Those will be changed during the gridsearch below.

In [ ]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', Lasso(max_iter=2000))])

In [ ]:
# now we create the grid with all the parameters that we would like to test

param_grid = {
    'preprocessor__numerical__imputer__strategy': ['mean', 'median'],
    'preprocessor__categorical__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__alpha': [10, 100, 200],
}

grid_search = GridSearchCV(clf, param_grid, cv=5, iid=False, n_jobs=-1, scoring='r2')

# cv=3 is the cross-validation
# no_jobs =-1 indicates to use all available cpus
# scoring='r2' indicates to evaluate using the r squared

# for more details in the grid parameters visit:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
# and now we train over all the possible combinations of the parameters above
grid_search.fit(X_train, y_train)

# and we print the best score over the train set
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_train, y_train)))

In [ ]:
# and find the best fit parameters like this
grid_search.best_params_

In [ ]:
# and finally let's check the performance over the test set ==> overfitting
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

# FeatureEngine

In [ ]:
# from feature-engine
from feature_engine import missing_data_imputers as mdi

#Feature-Engine captures the numerical variables automatically
# we call the imputer from feature-engine
# we specify the imputation strategy, median in this case

imputer = mdi.MeanMedianImputer(imputation_method='median')

In [ ]:
# we fit the imputer

imputer.fit(X_train)

In [ ]:
# we see that the imputer found the numerical variables to
# impute with the mean
imputer.variables

In [ ]:
# here we can see the mean assigned to each variable
imputer.imputer_dict_

In [ ]:
# feature-engine returns a dataframe

tmp = imputer.transform(X_train)
tmp.head()

In [ ]:
# let's check that the numerical variables don't
# contain NA any more

tmp[imputer.variables].isnull().mean()

### If you want to specify variables for specific technique

In [ ]:
# let's do mean imputation this time
# and let's do it over 2 of the 3 numerical variables

imputer = mdi.MeanMedianImputer(imputation_method='mean',
                                variables=['LotFrontage', 'MasVnrArea'])

imputer.fit(X_train)

### Using pipeline from sklearn

In [ ]:
pipe = Pipeline([
    ('median_imputer', mdi.MeanMedianImputer(imputation_method='median',
                                             variables = ['LotFrontage', 'GarageYrBlt'])),
     
    ('mean_imputer', mdi.MeanMedianImputer(imputation_method='mean',
                                          variables = ['MasVnrArea'])),
     ])

In [ ]:
pipe.fit(X_train)

In [ ]:
# let's transform the data with the pipeline
tmp = pipe.transform(X_train)

# let's check null values are gone
tmp.isnull().mean()

## Arbitrary method

In [ ]:
pipe = Pipeline([
    ('imputer_999', mdi.ArbitraryNumberImputer(arbitrary_number = -999,
                                             variables = ['LotFrontage', 'MasVnrArea'])),
     
    ('imputer_minus1', mdi.ArbitraryNumberImputer(arbitrary_number = -1,
                                          variables = ['GarageYrBlt'])),
     ])

## EndTail method

In [ ]:
pipe = Pipeline([
    ('imputer_skewed', mdi.EndTailImputer(distribution='skewed', tail='right',
                                          variables=['GarageYrBlt', 'MasVnrArea'])),

    ('imputer_gaussian', mdi.EndTailImputer(distribution='gaussian', tail='right',
                                            variables=['LotFrontage'])),
])

## Add MissingIndicator

In [ ]:
pipe = Pipeline([
    ('missing_ind', mdi.AddMissingIndicator()),
    
    ('imputer_mode', mdi.CategoricalVariableImputer(
        imputation_method='frequent', variables=['FireplaceQu', 'BsmtQual'])),
    
    ('imputer_median', mdi.MeanMedianImputer(imputation_method='median',
                                             variables=['LotFrontage', 'MasVnrArea', 'GarageYrBlt'])),
])

# Encoding

## One hot encoding

In [ ]:
# pandas
tmp = pd.get_dummies(X_train)
# The limitations:

#The train set contains 13 dummy features, whereas the test set contains 12 features. This occurred because there was no category T in cabin in the test set.

#This will cause problems if training and scoring models with scikit-learn, because predictors require train and test sets to be of the same shape.

In [ ]:
# sklearn. Limitations: returns an array and no variable names
# we create and train the encoder
encoder = OneHotEncoder(categories='auto',
                       drop='first', # to return k-1, use drop=false to return k dummies
                       sparse=False,
                       handle_unknown='error') # helps deal with rare labels

encoder.fit(X_train.fillna('Missing'))

# transform the train set
tmp = encoder.transform(X_train.fillna('Missing'))

pd.DataFrame(tmp).head()

# NEW: in latest release of Scikit-learn
# we can now retrieve the feature names as follows:

encoder.get_feature_names()

# we can go ahead and transfom the test set
# and then reconstitute it back to a pandas dataframe
# and add the feature names derived by OHE

tmp = encoder.transform(X_test.fillna('Missing'))

tmp = pd.DataFrame(tmp)
tmp.columns = encoder.get_feature_names()

In [ ]:
# featureEngine. BEST. Can choose the variables to one-hot encode
ohe_enc = OneHotCategoricalEncoder(
    top_categories=None,
    variables=['sex', 'embarked'], # we can select which variables to encode
    drop_last=True) # to return k-1, false to return k

#fit
ohe_enc.fit(X_train.fillna('Missing'))
#transform
tmp = ohe_enc.transform(X_train.fillna('Missing'))

# Note how feature-engine returns the dummy variables with their names, and drops the original variable, 
# leaving the dataset ready for further exploration or building machine learning models.

# Note how this encoder returns a variable cabin_T for the test set as well, even though this category is not 
# present in the test set. This allows the integration with Scikit-learn pipeline and scoring of test set by 
# the built algorithm..

## One-hot encoding of top categories

In [ ]:
# check cardinality of each variables

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' labels')

In [ ]:
# Good for high cardinal vars. better performance. Use feature-engine to keep the info of the ignored labels
ohe_enc = OneHotCategoricalEncoder(
    top_categories=10,  # you can change this value to select more or less variables
    # we can select which variables to encode
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'],
    drop_last=False)

# fit
ohe_enc.fit(X_train)

# in the encoder dict we can observe each of the top categories
# selected for each of the variables

ohe_enc.encoder_dict_

#transform on both train and test sets
X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

# let's explore the result
X_train.head()


#**Note**

#If the argument variables is left to None, then the encoder will automatically identify all categorical variables

#The encoder will not encode numerical variables. So if some of your numerical variables are in fact categories, you will need to re-cast them as object before using the encoder.

## Ordinal encoding

In [ ]:
# Using feature-engine
ordinal_enc = OrdinalCategoricalEncoder(
    encoding_method='arbitrary',
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])
#fit
ordinal_enc.fit(X_train)

# in the encoder dict we can observe the numbers
# assigned to each category for all the indicated variables
ordinal_enc.encoder_dict_

# Note, if there is a variable in the test set, for which the encoder doesn't have a number to assigned
# (the category was not seen in the train set), the encoder will return an error.

## Count/Frequency encoding

In [ ]:
# Using feature-engine. Pb : can lead to different labels have the same encoded value
count_enc = CountFrequencyCategoricalEncoder(
    encoding_method='count', # to do frequency ==> encoding_method='frequency'
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

count_enc.fit(X_train)

## Target guided encoding

In [ ]:
ordinal_enc = OrdinalCategoricalEncoder(
    # NOTE that we indicate ordered in the encoding_method, otherwise it assings numbers arbitrarily
    encoding_method='ordered',
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

# when fitting the transformer, we need to pass the target as well
# just like with any Scikit-learn predictor class
ordinal_enc.fit(X_train, y_train)

# transform
X_train = ordinal_enc.transform(X_train)
X_test = ordinal_enc.transform(X_test)


## Mean encoding

In [ ]:
# using feature-engine
mean_enc = MeanCategoricalEncoder(
    variables=['cabin', 'sex', 'embarked'])

# when fitting the transformer, we need to pass the target as well
# just like with any Scikit-learn predictor class
mean_enc.fit(X_train, y_train)

#transform
X_train = mean_enc.transform(X_train)
X_test = mean_enc.transform(X_test)

## Probability ratio encoding

In [ ]:
# Only for binary classification 
ratio_enc = WoERatioCategoricalEncoder(
    encoding_method = 'ratio',
    variables=['cabin', 'sex', 'embarked'])

# If the probability of target = 0 is zero for any category, the encoder will raise an error as the division 
# by zero is not defined.

## Weight of evidence encoding

In [ ]:
# Only for binary classification
woe_enc = WoERatioCategoricalEncoder(
    encoding_method = 'woe',
    variables=['cabin', 'sex', 'embarked'])

## Compare encoding methods performance on RF

In [ ]:
def get_OHE(df):

    df_OHE = pd.concat(
        [df[['pclass', 'age', 'sibsp', 'parch', 'fare']],
         pd.get_dummies(df[['sex', 'cabin', 'embarked']], drop_first=True)],
        axis=1)

    return df_OHE


X_train_OHE = get_OHE(X_train)
X_test_OHE = get_OHE(X_test)

In [ ]:
def categories_to_ordered(df_train, df_test, y_train, y_test):

    # make a temporary copy of the datasets
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # order categories according to target mean
        ordered_labels = df_train_temp.groupby(
            [col])['survived'].mean().sort_values().index

        # create the dictionary to map the ordered labels to an ordinal number
        ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

        # remap the categories  to these ordinal numbers
        df_train_temp[col] = df_train[col].map(ordinal_label)
        df_test_temp[col] = df_test[col].map(ordinal_label)

    # remove the target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_ordered, X_test_ordered = categories_to_ordered(
    X_train, X_test, y_train, y_test)

In [ ]:
# create a function to build random forests and compare performance in train and test set


def run_randomForests(X_train, X_test, y_train, y_test):

    rf = RandomForestClassifier(n_estimators=50, random_state=39, max_depth=3)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict_proba(X_train)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = rf.predict_proba(X_test)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [ ]:
# OHE
run_randomForests(X_train_OHE, X_test_OHE, y_train, y_test)

In [ ]:
# ordered labels
run_randomForests(X_train_ordered, X_test_ordered, y_train, y_test)

# Rare label encoding: first find rare variables

In [ ]:
def rare_encoding(X_train, X_test, variable, tolerance):

    X_train = X_train.copy()
    X_test = X_test.copy()

    # find the most frequent category
    frequent_cat = find_non_rare_labels(X_train, variable, tolerance)

    # re-group rare labels
    X_train[variable] = np.where(X_train[variable].isin(
        frequent_cat), X_train[variable], 'Rare')
    
    X_test[variable] = np.where(X_test[variable].isin(
        frequent_cat), X_test[variable], 'Rare')

    return X_train, X_test

In [ ]:
from feature_engine.categorical_encoders import RareLabelCategoricalEncoder

# let's divide into train and test set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1), # predictors
    data.SalePrice, # target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

# Rare value encoder
rare_encoder = RareLabelCategoricalEncoder(
    tol=0.05,  # minimal percentage to be considered non-rare
    n_categories=4, # minimal number of categories the variable should have to re-cgroup rare categories
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd',
               'MasVnrType', 'ExterQual', 'BsmtCond'] # variables to re-group
)  

# fit
rare_encoder.fit(X_train.fillna('Missing'))

# transform
X_train = rare_encoder.transform(X_train.fillna('Missing'))
X_test = rare_encoder.transform(X_test.fillna('Missing'))

# Variable transformation

## Assess normality

In [ ]:
# plot the histograms to have a quick look at the variable distribution
# histogram and Q-Q plots

def diagnostic_plots(df, variable):
    
    # function to plot a histogram and a Q-Q plot
    # side by side, for a certain variable
    
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    df[variable].hist(bins=30)

    plt.subplot(1, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)

    plt.show()
    
diagnostic_plots(data, 'GrLivArea')

In [ ]:
### Logarithmic transformation

data['GrLivArea_log'] = np.log(data['GrLivArea'])

diagnostic_plots(data, 'GrLivArea_log')


### Square root transformation
data['GrLivArea_sqr'] = data['GrLivArea']**(1/2) 

# np.power(data['GrLivArea'], 1/2), np.sqrt(data['GrLivArea'])

diagnostic_plots(data, 'GrLivArea_sqr')


### Exponential transformation
data['GrLivArea_exp'] = data['GrLivArea']**(1/1.5) # you can vary the exponent as needed

# np.power(data['GrLivArea'], any exponent we want)

diagnostic_plots(data, 'GrLivArea_exp')



### BOX-COX Transformation
data['GrLivArea_boxcox'], param = stats.boxcox(data['GrLivArea']) 

print('Optimal λ: ', param)

diagnostic_plots(data, 'GrLivArea_boxcox')


In [ ]:
# Logarithmic transformer

# create a log transformer

transformer = FunctionTransformer(np.log, validate=True)

# transform all the numerical and positive variables. sklearn doesn't accept missing values as input so need fillna

data_t = transformer.transform(data[cols].fillna(1))

# Scikit-learn returns NumPy arrays, so capture in dataframe
# note that Scikit-learn will return an array with
# only the columns indicated in cols

data_t = pd.DataFrame(data_t, columns = cols)



In [ ]:
# Yeo-Johnson

# Yeo-Johnson is an adaptation of Box-Cox that can also be used in negative value variables

# call the transformer
transformer = PowerTransformer(method='yeo-johnson', standardize=False)

# learn the lambda from the train set
transformer.fit(data[cols].fillna(1))

# transform the data
data_t = transformer.transform(data[cols].fillna(1))

# capture data in a dataframe
data_t = pd.DataFrame(data_t, columns = cols)

# Discretisation

In [ ]:
# Equal-width with feature-engine
from feature_engine.discretisers import EqualWidthDiscretiser

# Let's separate into train and test set
X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare']],
    data['survived'],
    test_size=0.3,
    random_state=0)

# replace NA in both  train and test sets
X_train['age'] = impute_na(data, 'age')
X_test['age'] = impute_na(data, 'age')
X_train['fare'] = impute_na(data, 'fare')
X_test['fare'] = impute_na(data, 'fare')

# with feature engine we can automate the process for many variables
disc = EqualWidthDiscretiser(bins=10, variables = ['age', 'fare'])

disc.fit(X_train)

# in the binner dict, we can see the limits of the intervals. For age
# the value increases aproximately 7 years from one bin to the next.
# for fare it increases in around 50 dollars from one interval to the 
# next, but it increases always the same value, aka, same width.

disc.binner_dict_

# transform train and text
train_t = disc.transform(X_train)
test_t = disc.transform(X_test)


In [ ]:
# Equal-frequency. Boundaries represent quantiles --> each interval has the same amount of observations. 
# Useful for skwed variables cuz it spreads the observations over the different intervals equally
# The width of intervals is not the same --> the nb of observations matters
# Same process than equal-width

# with feature engine we can automate the process for many variables
# in one line of code

disc = EqualFrequencyDiscretiser(q=10, variables = ['age', 'fare'], return_object = True # True to return cat vars)

disc.fit(X_train)
                                 
# transform train and text
train_t = disc.transform(X_train)
test_t = disc.transform(X_test)

In [ ]:
# let's explore if the bins have a linear relationship
# with the target:

pd.concat([train_t, y_train], axis=1).groupby('age')['survived'].mean().plot()
plt.ylabel('mean of survived')

# Then use discreted variables as cat vars and ordinalcategorical to obtain a monotonic relationship (re-order the order of bins)
enc = OrdinalCategoricalEncoder(encoding_method = 'ordered')

enc.fit(train_t, y_train)

train_t = enc.transform(train_t)
test_t = enc.transform(test_t)

# in the map, we map bin to position. Run this code to know the mapping (already done)
enc.encoder_dict_

# Check
pd.concat([train_t, y_train], axis=1).groupby('age')['survived'].mean().plot()
plt.ylabel('mean of survived')

In [ ]:
# Discretization using decision tree: find the optimal bins and 2 steps in one : discretisation and encoding same time
# Good cuz create a monotonic relationship
from feature_engine.discretisers import DecisionTreeDiscretiser

# set up the decision tree discretiser indicating:
# cross-validation number (cv)
# how to evaluate model performance (scoring)
# the variables we want to discretise (variables)
# whether it is a target for regression or classification
# and the grid with the parameters we want to test

treeDisc = DecisionTreeDiscretiser(cv=10, scoring='accuracy',
                                   variables=['age', 'fare'],
                                   regression=False,
                                   param_grid={'max_depth': [1, 2, 3],
                                              'min_samples_leaf':[10,4]})

treeDisc.fit(X_train, y_train)

# we can inspect the tree for age to find the best params
treeDisc.binner_dict_['age'].best_params_

# and the performance obtained on the train set while fitting: print the score (accuracy for the best tree)
treeDisc.scores_dict_['age']

# let's transform the data
train_t = treeDisc.transform(X_train)
test_t = treeDisc.transform(X_test)

# let's inspect how many bins we found
train_t['age'].unique()

# monotonic relationship with target: train set
pd.concat([train_t, y_train], axis=1).groupby(['age'])['survived'].mean().plot()
plt.title('Monotonic relationship between discretised Age and target')
plt.ylabel('Survived')

# Outliers handling: only remove from train set

In [1]:
# Detect outliers using IQR : upper limit = Q3 + IQR*1.5 and lower limit = Q1 - IQR*1.5. NB : IQR = Q3 - Q1
def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

# find limits for variable RM
RM_upper_limit, RM_lower_limit = find_skewed_boundaries(df, 'RM', 1.5)
RM_upper_limit, RM_lower_limit

# let's flag the outliers in the data set
outliers_RM = np.where(df['RM'] > RM_upper_limit, True,
                       np.where(df['RM'] < RM_lower_limit, True, False))

In [ ]:
# Trimming (remove outliers)
df_trimmed = df.loc[~outliers_RM, ]

df.shape, df_trimmed.shape # Check shape to see how many outliers we removed. No more than 5%

In [ ]:
# Capping (fix limit values)
# Now let's replace the outliers by the maximum and minimum limit. Replace and not flag

df['RM']= np.where(df['RM'] > RM_upper_limit, RM_upper_limit,
                       np.where(df['RM'] < RM_lower_limit, RM_lower_limit, df['RM']))

# Features scaling: important except for trees

In [ ]:
# Standardization
from sklearn.preprocessing import StandardScaler
# let's separate the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(data.drop('MEDV', axis=1),
                                                    data['MEDV'],
                                                    test_size=0.3,
                                                    random_state=0)

# standardisation: with the StandardScaler from sklearn

# set up the scaler
scaler = StandardScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)
# transform train and test sets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# let's transform the returned NumPy arrays to dataframes
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
# MinMax. Sensitive to outliers
from sklearn.preprocessing import MinMaxScaler
# set up the scaler
scaler = MinMaxScaler()

# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)

# transform train and test sets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Robust scaling. Good if data shows outliers
from sklearn.preprocessing import RobustScaler
# set up the scaler
scaler = RobustScaler()


# Mixed variables

In [ ]:
#1) Variables that contain number and strings but the value is number or string. 
# So we extract the num part and the cat part into two different vars
# extract numerical part
data['var_num'] = pd.to_numeric(data["mixed_var"],
                                              errors='coerce',
                                              downcast='integer')

# extract categorical part
data['var_cat'] = np.where(data['var_num'].isnull(),
                                           data['mixed_var'], 
                                           np.nan)


In [ ]:
#2) Variables whose values contain number and srings. Like ticket : 145C25RE
# Same technique: extract num and cat parts. But not optimal strategy, be careful

# let's extract the numerical and categorical part for ticket
# the variable ticket is extremely dirty, so there is only so much that we
# can do, but here are some ideas:

# extract the last bit of ticket as number
data['ticket_num'] = data['ticket'].apply(lambda s: s.split()[-1])
data['ticket_num'] = pd.to_numeric(data['ticket_num'],
                                   errors='coerce',
                                   downcast='integer')

# extract the first part of ticket as category
data['ticket_cat'] = data['ticket'].apply(lambda s: s.split()[0])
data['ticket_cat'] = np.where(data['ticket_cat'].str.isdigit(), np.nan,
                              data['ticket_cat'])

data[['ticket', 'ticket_num', 'ticket_cat']].head(20)

# Date and time variables

## Date

In [ ]:
import datetime
# Convert into date format (originally string format). 2 vars here
data['issue_dt'] = pd.to_datetime(data['date_issued'])
data['last_pymnt_dt'] = pd.to_datetime(data['date_last_payment'])


# Extracting week of year from date, varies from 1 to 52
data['issue_dt_week'] = data['issue_dt'].dt.week

# Extracting month from date - 1 to 12
data['issue_dt_month'] = data['issue_dt'].dt.month

# Extract quarter from date variable - 1 to 4
data['issue_dt_quarter'] = data['issue_dt'].dt.quarter

# extract semester
data['issue_dt_semester'] = np.where(data['issue_dt_quarter'].isin([1,2]), 1, 2)

# extract year 
data['issue_dt_year'] = data['issue_dt'].dt.year

# day - numeric from 1-31
data['issue_dt_day'] = data['issue_dt'].dt.day

# day of the week - from 0 to 6
data['issue_dt_dayofweek'] = data['issue_dt'].dt.dayofweek

# day of the week - name
data['issue_dt_dayofweek'] = data['issue_dt'].dt.day_name()

# isweekend. ?
data['issue_dt_is_weekend'] = np.where(data['issue_dt_dayofweek'].isin(['Sunday', 'Saturday']), 1,0)

# Calculate the delta between two dates
(data['last_pymnt_dt'] - data['issue_dt']).dt.days

# calculate number of months passed between 2 dates
data['months_passed'] = (data['last_pymnt_dt'] - data['issue_dt']) / np.timedelta64(1, 'M')
data['months_passed'] = np.round(data['months_passed'],0)

## Time

In [ ]:
import datetime
# Extract time in a datetime format
df['time'] = df['date'].dt.time

# Extrat hour, minute and second
df['hour'] = df['date'].dt.hour
df['min'] = df['date'].dt.minute
df['sec'] = df['date'].dt.second

# Recapitulation: Pipeline 

In [ ]:
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for the model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso # The model to use
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# for feature engineering
from sklearn.preprocessing import StandardScaler
from feature_engine import missing_data_imputers as mdi
from feature_engine import discretisers as dsc
from feature_engine import categorical_encoders as ce

pd.pandas.set_option('display.max_columns', None)

In [ ]:
# Assembling a feature engineering pipeline
# First, make a list of the num and cat variables
categorical = [var for var in df.columns if df[var].dtype=='O']
numerical = [var for var in df.columns if df[var].dtype!='O']
# list of variables that contain year information for num vars that are not treated as num
year_vars = [var for var in numerical if 'Yr' in var or 'Year' in var]

# find discrete variables : To identify discrete variables, select from all the numerical ones, 
# those that contain a finite and small number of distinct values.
discrete = []

for var in numerical:
    if len(data[var].unique()) < 20 and var not in year_vars:
        print(var, ' values: ', data[var].unique())
        discrete.append(var)
print()
print('There are {} discrete variables'.format(len(discrete)))

In [ ]:
# Let's separate into train and test set
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Id', 'SalePrice'], axis=1),
                                                    data['SalePrice'],
                                                    test_size=0.1,
                                                    random_state=0)


In [ ]:
# I will treat discrete variables as if they were categorical
# to treat discrete as categorical using Feature-engine
# we need to re-cast them as object

X_train[discrete] = X_train[discrete].astype('O')
X_test[discrete] = X_test[discrete].astype('O')

In [ ]:
# pipeline
house_pipe = Pipeline([

    # missing data imputation - section 4
    ('missing_ind', mdi.AddNaNBinaryImputer(  # add missing indicator
        variables=['LotFrontage', 'MasVnrArea',  'GarageYrBlt'])),
    ('imputer_num', mdi.MeanMedianImputer(imputation_method='median',  # replace MV by the median
                                          variables=['LotFrontage', 'MasVnrArea',  'GarageYrBlt'])),
    ('imputer_cat', mdi.CategoricalVariableImputer(variables=categorical)),  # add string 'missing' to all cat var with MV

    # categorical encoding - section 6
    ('rare_label_enc', ce.RareLabelCategoricalEncoder(
        tol=0.05, n_categories=6, variables=categorical+discrete)),
    ('categorical_enc', ce.OrdinalCategoricalEncoder(
        encoding_method='ordered', variables=categorical+discrete)),

    # discretisation + encoding - section 8
    ('discretisation', dsc.EqualFrequencyDiscretiser(
        q=5, return_object=True, variables=numerical)), # Create 5 intervals
    ('encoding', ce.OrdinalCategoricalEncoder(
        encoding_method='ordered', variables=numerical)), # encode results intervals reorder to create monotonic

    # feature Scaling - section 10
    ('scaler', StandardScaler()), # scale cuz we use a linear model
    
    # regression
    ('model', Lasso(random_state=0))
])

In [ ]:
# let's fit the pipeline
house_pipe.fit(X_train, y_train)

# let's get the predictions
X_train_preds = house_pipe.predict(X_train)
X_test_preds = house_pipe.predict(X_test)

In [ ]:
# check model performance:

print('train mse: {}'.format(mean_squared_error(y_train, X_train_preds)))
print('train rmse: {}'.format(sqrt(mean_squared_error(y_train, X_train_preds))))
print('train r2: {}'.format(r2_score(y_train, X_train_preds)))
print()
print('test mse: {}'.format(mean_squared_error(y_test, X_test_preds)))
print('test rmse: {}'.format(sqrt(mean_squared_error(y_test, X_test_preds))))
print('test r2: {}'.format(r2_score(y_test, X_test_preds)))

In [ ]:
# let's explore the importance of the features
# the importance is given by the absolute value of the coefficient
# assigned by the Lasso

importance = pd.Series(np.abs(house_pipe.named_steps['lasso'].coef_))
importance.index = list(final_columns)+['LotFrontage_na', 'MasVnrArea_na',  'GarageYrBlt_na']
importance.sort_values(inplace=True, ascending=False)
importance.plot.bar(figsize=(18,6))

# Add cross validation

In [ ]:
# Pipeline
titanic_pipe = Pipeline([

    # missing data imputation - section 4
    ('imputer_num',
     mdi.ArbitraryNumberImputer(arbitrary_number=-1,
                                variables=['age', 'fare', 'cabin_num'])),  # impute with value -1
    ('imputer_cat',
     mdi.CategoricalVariableImputer(variables=['embarked', 'cabin_cat'])),

    # categorical encoding - section 6
    ('encoder_rare_label',
     ce.RareLabelCategoricalEncoder(tol=0.01,
                                    n_categories=2,
                                    variables=['embarked', 'cabin_cat'])),
    ('categorical_encoder',
     ce.OrdinalCategoricalEncoder(encoding_method='ordered',
                                  variables=['cabin_cat', 'sex', 'embarked'])),

    # Gradient Boosted machine
    ('gbm', GradientBoostingClassifier(random_state=0))
])

In [ ]:
# now we create the grid with all the parameters that we would like to test

param_grid = {
    # try different feature engineering parameters
    'imputer_num__arbitrary_number': [-1, 99],
    'encoder_rare_label__tol': [0.1, 0.2],  # Test different levels of rare level: 10% or 20%
    'categorical_encoder__encoding_method': ['ordered', 'arbitrary'],
    
    # try different gradient boosted tree model paramenters
    'gbm__max_depth': [None, 1, 3],
}


# now we set up the grid search with cross-validation
grid_search = GridSearchCV(titanic_pipe, param_grid,
                           cv=5, n_jobs=-1, scoring='roc_auc')

# cv=5 is the cross-validation steps
# no_jobs =-1 indicates to use all available cpus
# scoring='roc-auc' indicates to evaluate the model performance with the roc-auc

In [ ]:
# and now we train over all the possible combinations of the parameters above
grid_search.fit(X_train, y_train)

# and we print the best score over the train set
print(("best roc-auc from grid search: %.3f"
       % grid_search.score(X_train, y_train)))

In [ ]:
# we can print the best estimator parameters like this
grid_search.best_estimator_

In [ ]:
# and find the best fit parameters like this
grid_search.best_params_

In [ ]:
# and finally let's check the performance over the test set
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

In [ ]:
# let's explore the importance of the features

importance = pd.Series(grid_search.best_estimator_['gbm'].feature_importances_)
importance.index = data.drop('survived', axis=1).columns
importance.sort_values(inplace=True, ascending=False)
importance.plot.bar(figsize=(12,6))